In [ ]:
# https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-4.%20kogpt2_nsmc_tpu.ipynb

In [ ]:
# !pip install transformers

In [ ]:
import transformers
transformers.__version__

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# logging.basicConfig(level=logging.ERROR)

In [ ]:
# 데이터 불러오기
data = pd.read_csv('./data/spam.csv')

data['label'].loc[data['label']=='ham'] = 0
data['label'].loc[data['label']=='spam'] = 1

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=.2)

In [ ]:
print('훈련용 리뷰 개수 :', len(train_data)) # 훈련용 리뷰 개수 출력

In [ ]:
print('테스트용 리뷰 개수 :', len(test_data)) # 테스트용 리뷰 개수 출력

In [ ]:
train_data[:5] # 상위 5개 출력

In [ ]:
test_data[:5]

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data.reset_index(drop=True)
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
test_data = test_data.reset_index(drop=True)
print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
print(len(train_data))

In [ ]:
print(len(test_data))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
max_seq_len = 128

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['text'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
test_X, test_y = convert_examples_to_features(test_data['text'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
# 최대 길이: 128
input_id = train_X[0]
label = train_y[0]

In [ ]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=2, batch_size=8, validation_split=0.2)

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=8)
print("test loss, test acc: ", results)

In [ ]:
# 데이터 불러오기
spam = pd.read_csv('./data/spam_test_text.csv')

In [ ]:
test_X, _ = convert_examples_to_features(spam['text'], spam['id'], max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
pred = model.predict(test_X)

In [ ]:
pred = np.where(pred>0.5, 'spam', 'ham')

In [ ]:
ids = list(range(len(pred)))

In [ ]:
results = pd.DataFrame({'id': ids, 'label': pred.reshape(-1)})

In [ ]:
results.head()

In [ ]:
results.to_csv('kogpt2_spam_v1.csv', index=False)